In [ ]:
from common import *

use_hvplot()

In [ ]:
client = start_cluster(4, 20)
client_ip_and_port(client)

# Precipitation input comparison between iSnobal and CBRFC Snow-17 (HRRR F06)

## CBRFC

Precip is average for each zone

In [ ]:
CBRFC_CSV = pd.read_csv(
    CBRFC_DIR / 'precip_Alec2.csv',
    parse_dates=True,
    index_col=0,
    header=0,
    names=['Lower', 'Middle', 'Upper', 'Average'],
    dtype={'Lower': np.float64, 'Middle': np.float64, 'Upper': np.float64},
)
CBRFC_CSV.Lower *= INCH_TO_MM
CBRFC_CSV.Middle *= INCH_TO_MM
CBRFC_CSV.Upper *= INCH_TO_MM

## iSnobal 

In [ ]:
wy_precip = xr.open_mfdataset(
    (SNOBAL_DIR / 'wy20*' / 'erw/*/smrf_20*.nc').as_posix(),
    parallel=True,
).resample(**RESAMPLE_1_DAY_OPTS).sum()

wy_precip.coords['hru'] = (('y', 'x'), cbrfc_zones())
wy_precip.coords['aspect'] = (('y', 'x'), aspect_classes())

In [ ]:
precip_time = {}
start_year = 2017

for y_index in range(0, 4):
    year = start_year + y_index
    time=slice(f"{year}-10-01", f"{year + 1}-09-30")
    precip_time[str(year + 1)] = wy_precip.sel(time=time).precip.sum('time').compute()

In [ ]:
hlf = wy_precip.where(wy_precip.hru == ALEC2HLF).precip.mean(("x", "y")).compute()
hmf = wy_precip.where(wy_precip.hru == ALEC2HMF).precip.mean(("x", "y")).compute()
huf = wy_precip.where(wy_precip.hru == ALEC2HUF).precip.mean(("x", "y")).compute()

### Treating all iSnobal values of less than 1 mm as no precipitation 

In [ ]:
hlf[hlf < 1] = 0
hmf[hmf < 1] = 0
huf[huf < 1] = 0

## Precipitation 

In [ ]:
huf.rolling(time=7, center=True).mean().hvplot(label='Upper (iSnobal)', color='indigo', alpha=0.6). \
    opts(
        title='iSnobal vs Snow-17 Precipitation - 7-day moving average', ylabel='Precipitation (mm)',
        yformatter='%d',
        width=1280, height=720
    ) * \
CBRFC_CSV.Upper.rolling(7, center=True).mean().plot(color='indigo', line_dash='dashed', alpha=0.6) * \
hmf.rolling(time=7, center=True).mean().hvplot(label='Middle (iSnobal)', color='teal', alpha=0.6) * \
CBRFC_CSV.Middle.rolling(7, center=True).mean().plot(color='teal', line_dash='dashed', alpha=0.6) * \
hlf.rolling(time=7, center=True).mean().hvplot(label='Lower (iSnobal)', color='gold', alpha=0.6) * \
CBRFC_CSV.Lower.rolling(7, center=True).mean().plot(color='gold', line_dash='dashed', alpha=0.6)

In [ ]:
huf.hvplot(label='Upper (iSnobal)', color='indigo', alpha=0.6). \
    opts(
        title='iSnobal vs Snow-17 Precipitation', ylabel='Precipitation (mm)',
        yformatter='%d',
        width=1280, height=720
    ) * \
CBRFC_CSV.Upper.plot(color='indigo', line_dash='dashed', alpha=0.6) * \
hmf.hvplot(label='Middle (iSnobal)', color='teal', alpha=0.6) * \
CBRFC_CSV.Middle.plot(color='teal', line_dash='dashed', alpha=0.6) * \
hlf.hvplot(label='Lower (iSnobal)', color='gold', alpha=0.6) * \
CBRFC_CSV.Lower.plot(color='gold', line_dash='dashed', alpha=0.6)

In [ ]:
(CBRFC_CSV.Upper / huf).replace([np.nan, np.inf], 0).hvplot(label='Upper', color='indigo', alpha=0.6). \
    opts(
        title='Snow-17/iSnobal Precipitation Ratios',
        yformatter='%d',
        width=1280, height=720
    ) * \
(CBRFC_CSV.Middle / hmf).replace([np.nan, np.inf], 0).hvplot(label='Middle', color='teal', alpha=0.6) * \
(CBRFC_CSV.Lower / hlf).replace([np.nan, np.inf], 0).hvplot(label='Lower', color='gold', alpha=0.6)

### iSnobal 

In [ ]:
huf.hvplot(label='Upper(iSnobal)', color='indigo', alpha=0.6). \
    opts(
        title='iSnobal Precipitation', ylabel='Precipitation (mm)',
        yformatter='%d',
        width=1280, height=640
    ) * \
hmf.hvplot(label='Middle (iSnobal)', color='teal', alpha=0.6) * \
hlf.hvplot(label='Lower (iSnobal)', color='gold', alpha=0.6)

### CBRFC 

In [ ]:
CBRFC_CSV.Upper.plot(color='indigo', line_dash='dashed', alpha=0.6). \
    opts(title='Snow-17 CBRFC Precipitation', ylabel='Precipitation (mm)', width=1280, height=640) * \
CBRFC_CSV.Middle.plot(color='teal', line_dash='dashed', alpha=0.6) * \
CBRFC_CSV.Lower.plot(color='gold', line_dash='dashed', alpha=0.6)

## By Aspect 

In [ ]:
def drop_nan(array):
    aspect_data = []

    for aspect in range(1, 9):
        aspect_array = array.where(array.aspect == aspect).where(array.hru != 0).values.flatten()
        mask = np.isnan(aspect_array)
        aspect_data.append(aspect_array[~mask])
    
    return aspect_data

def plot_aspect_mpl(precip, year):
    fig, (ax) = plt.subplots(ncols=1, sharey=True, dpi=300, gridspec_kw={'wspace': 0})

    ax.boxplot(
        drop_nan(precip),
        whis=(5, 95),
        showmeans=True,
        # positions=box_position,
        notch=True,
        patch_artist=True,
        labels=['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW'],
        medianprops=dict(
            color='black',
            linestyle=':',
            linewidth=0.75
        ),
        meanprops=dict(
            marker='D', 
            markeredgecolor='black',
            markerfacecolor='firebrick'
        ),
        boxprops=dict(
            linewidth=0.75,
            # facecolor=color,
            alpha=0.5,
            edgecolor='black',
        ),
        showfliers=False,
    )
    ax.set_ylabel('Precipitation (mm)')
    ax.set_xlabel('Aspect')
    plt.grid(axis='y', color='0.95')
    plt.title(year)

In [ ]:
start_year = 2018

for y_index in range(0, 3):
    year = start_year + y_index
                     
    plot_aspect_mpl(precip_time[str(year)], year)

## Yearly statistics 

### Total precipitation 

In [ ]:
stats = np.zeros([9,4])
start_year = 2017

for y_index in range(0, stats.shape[1]):
    year = start_year + y_index
    time=slice(f"{year}-10-01", f"{year + 1}-09-30")
    
    stats[0, y_index] = huf.sel(time=time).sum()
    stats[1, y_index] = CBRFC_CSV.Upper[time].sum()
    stats[2, y_index] = stats[0, y_index] / stats[1, y_index]

    stats[3, y_index] = hmf.sel(time=time).sum()
    stats[4, y_index] = CBRFC_CSV.Middle[time].sum()
    stats[5, y_index] = stats[3, y_index] / stats[4, y_index]
    
    stats[6, y_index] = hlf.sel(time=time).sum()
    stats[7, y_index] = CBRFC_CSV.Lower[time].sum()
    stats[8, y_index] = stats[6, y_index] / stats[7, y_index]

In [ ]:
results = pd.DataFrame(
    stats, 
    columns=range(2018, 2022), 
    index=pd.MultiIndex.from_product([
        ['Upper', 'Middle', 'Lower'], 
        ['iSNOBAL', 'Snow-17', 'iSNOBAL/Snow-17']
    ])
)

headers = {
    'selector': 'th:not(.index_name)',
    'props': 'font-size: 16pt;'
}

ratios = pd.IndexSlice[:, 'iSNOBAL/Snow-17', :]

results.style.set_table_styles(
    [headers]
).set_properties(
    **{'font-size': '12pt'}
).format('{:.2f} mm').format('{:.2%}', subset=ratios)

In [ ]:
box_style = dict(
    ylabel='Percent (%)',
    xlabel='Year',
    width=640, 
    **BOKEH_FONT,
)

def plot_year(year):
    time=slice(f"{year-1}-10-01", f"{year}-09-30")
    box_opts=dict(ylim=(-2, 12), violin_fill_alpha=0.4)

    box_stats = hv.Violin(
        (CBRFC_CSV.Upper / huf).replace([np.inf], np.nan)[time].values, 
        group=str(year), label="Upper"
    ).opts(violin_fill_color='indigo', **box_opts, **box_style) * \
    hv.Violin(
        (CBRFC_CSV.Middle / hmf).replace([np.inf], np.nan)[time].values, 
        group=str(year), label="Middle"
    ).opts(violin_fill_color='teal', **box_opts, **box_style) * \
    hv.Violin(
        (CBRFC_CSV.Lower / hlf).replace([np.inf], np.nan)[time].values, 
        group=str(year), label="Lower"
    ).opts(violin_fill_color='gold', **box_opts, **box_style)

    return box_stats

(
    hv.Layout(plot_year(2018)) + \
    hv.Layout(plot_year(2019)) + \
    hv.Layout(plot_year(2020)) + \
    hv.Layout(plot_year(2021))
).cols(2).opts(title="Ratio Snow-17/iSnobal", shared_axes=False, fontsize=14)

In [ ]:
def plot_year(isnobal, zone, color):
    box_stats = []
    for year in range(2018, 2022):
        time=slice(f"{year-1}-10-01", f"{year}-09-30")
        box_opts = dict(ylim=(-0.5, 8), box_fill_alpha=0.4)

        box_stats.append(
            hv.BoxWhisker(
                (CBRFC_CSV[zone] / isnobal).replace([np.inf], np.nan)[time].values, 
                group=zone, label=str(year)
            ).opts(box_fill_color=color, **box_opts, **box_style)
        )

    return box_stats[0] * box_stats[1] * box_stats[2] * box_stats[3]

(
    hv.Layout(plot_year(huf, "Upper", 'indigo')) + \
    hv.Layout(plot_year(hmf, "Middle", 'teal')) + \
    hv.Layout(plot_year(hlf, "Lower", 'gold'))
).cols(1).opts(title="Ratio Snow-17/iSnobal", shared_axes=False,  fontsize=14)

In [ ]:
client.shutdown()

In [ ]:
def stats_by_group(isnobal, zone):
    box_stats = []
    for year in range(2018, 2022):
        time=slice(f"{year-1}-10-01", f"{year}-09-30")
        box_opts = dict(ylim=(-0.5, 8), box_fill_alpha=0.4)

        box_stats.append(
            (CBRFC_CSV[zone] / isnobal).replace([np.inf], np.nan).dropna()[time].values
        )
    
    return box_stats

In [ ]:
def plot_mpl(data):
    fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(9, 3.2), dpi=300, sharey=True)
    fig.set_facecolor('white')
    
    plot_zone(ax1, data[0], 'Lower', 'gold')
    plot_zone(ax2, data[1], 'Middle', 'teal')
    plot_zone(ax3, data[2], 'Upper', 'indigo')
    
    ax1.set_ylim(-0.5, 8.5)
    ax1.set_ylabel('Daily Precipitation Difference (%)')


def plot_zone(ax, stats, title, color):
    box_position = [0.2, 0.5, 0.8, 1.1]
    x_labels = ['2018', '2019', '2020', '2021']
    bdata = ax.boxplot(
        stats,
        whis=(5, 95),
        positions=box_position,
        notch=True,
        patch_artist=True,
        labels=x_labels,
        medianprops=dict(
            color='black',
            linestyle=':',
            linewidth=0.75
        ),
        boxprops=dict(
            linewidth=0.75,
            facecolor=color,
            alpha=0.3,
            edgecolor='black',
        ),
        flierprops=dict(
            marker='o', 
            markerfacecolor='black', 
            markersize=3,
            markeredgecolor='none'
        )
    )

    ax.set_xlim(0, 1.3)

    ax.set_title(title, fontstyle='italic');

In [ ]:
plot_mpl([stats_by_group(hlf, 'Lower'), stats_by_group(hmf, 'Middle'), stats_by_group(huf, 'Upper')])